In [1]:
%pip install azure-ai-ml azure-identity

Note: you may need to restart the kernel to use updated packages.


In [2]:
# deployment
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

In [3]:
# enter details of your Azure Machine Learning workspace
subscription_id = "1c623b04-20a0-4b59-897e-ceef40c48e27"
resource_group = "OasisAi_group"
workspace = "oasisAi"

In [4]:
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [27]:
import datetime

endpoint_name = "lugtts-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name, 
    description="This is the endpoint for inferencing text to speech in luganda",
    auth_mode="key"
)

In [67]:
model = Model(path="Conda.yaml")
env = Environment(
    conda_file="Conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
)

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="Inference", scoring_script="Score.py"
    ),
    instance_type="STANDARD_NC6S_V3",
    instance_count=1,
)

In [68]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)

Updating local endpoint (lugtts-endpt-09060913290567) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32800/score', 'openapi_uri': None, 'name': 'lugtts-endpt-09060913290567', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiSpeechToText'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f57a8433ac0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [69]:
ml_client.online_deployments.begin_create_or_update(
    deployment=blue_deployment, local=True
)

Updating local deployment (lugtts-endpt-09060913290567 / blue) .
Building Docker image from Dockerfile
Step 1/6 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04
 ---> a79caa0393e2
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 5286c4c3769f
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> 0e4d45b04cdc
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 543a8d7da2f9
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> 64097cdae03c
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> 338e497468c1
Successfully built 338e497468c1
Successfully tagged lugtts-endpt-09060913290567:blue

Starting up endpoint.....Done (0m 30s)


ManagedOnlineDeployment({'private_network_connection': None, 'provisioning_state': 'Succeeded', 'endpoint_name': 'lugtts-endpt-09060913290567', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiSpeechToText'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f57a99dc970>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '68f10e26af33c7674813d7aeb9e06991', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiSpeechToText'), 'creation_context': None, 'serialize':

In [70]:
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=endpoint_name, local=True, lines=500
)

"2023-09-06T10:14:59,888720714+00:00 - rsyslog/run \r\n2023-09-06T10:14:59,891979275+00:00 - nginx/run \r\n2023-09-06T10:14:59,893132996+00:00 - gunicorn/run \r\n2023-09-06T10:14:59,894736726+00:00 | gunicorn/run | \r\n2023-09-06T10:14:59,896267955+00:00 | gunicorn/run | ###############################################\r\n2023-09-06T10:14:59,897846084+00:00 | gunicorn/run | AzureML Container Runtime Information\r\n2023-09-06T10:14:59,899429014+00:00 | gunicorn/run | ###############################################\r\n2023-09-06T10:14:59,901108445+00:00 | gunicorn/run | \r\n2023-09-06T10:15:00,398893566+00:00 | gunicorn/run | \r\n2023-09-06T10:15:00,401678118+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20230227.v3\r\n2023-09-06T10:15:00,403029843+00:00 | gunicorn/run | \r\n2023-09-06T10:15:00,404412969+00:00 | gunicorn/run | \r\n2023-09-06T10:15:00,405740494+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/inf-conda

In [61]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32797/score', 'openapi_uri': None, 'name': 'lugtts-endpt-09060913290567', 'description': 'This is the endpoint for inferencing text to speech in luganda', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': None, 'Resource__source_path': None, 'base_path': PosixPath('/mnt/batch/tasks/shared/LS_root/mounts/clusters/oasisaitts/code/Users/2100800280/OasisAiSpeechToText'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7f57a84de6b0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [72]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="sample-request.json",
    local=True,
)

'{"result": "namatugoga nawulira nga ekigambo gwo ekikuvaako ennyuma nga kyogera nti nalyo ekkubo mu litambula munaakyamirga ku mukono ogwa ddyo era bwe munaakyamira nga ku gwa kkono nga tutunuulira yesu yekka omukulu okukkiriza kwaffe omutuukiriza waakwo olwo essanyu eriteeka mu maaso agumiikiriza omusaalaba nga anyuma ensonyi atuula ku mukono ogwa ddyo ogwa ntebe ya katonda"}'